## HUMAN_9606_idmapping_2018_08_01download.dat
## UniprotKB_isoforms_canonical_filtered_homosapiens_9606_2018_06release.fasta

In [1]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv
import argparse
import Bio
from Bio import SeqIO
from ast import literal_eval # for mismap_score func
import difflib # diff btw 2 strings
from statistics import mean
import jellyfish
import textdistance
from ast import literal_eval # for mismap_score func

In [2]:
sys.path.append("/Users/mariapalafox/Desktop/Toolbox/")
from all_funx import *

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [4]:
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

# set path
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CANONICAL_number/")

In [5]:
print(os.listdir())

['prism_hist_18432ccds_isoformCount.csv', 'hist_canonical_isoformNum_all_stableEntries_multiIsoformFiltered_diffInISOnum_1_10628.csv', 'prism_hist_2507labeled_canonicalIDnumber.csv', 'UniprotKB_isoforms_canonical_filtered_homosapiens_9606_2018_06release.fasta', 'IDmapping_2019_version', 'uniprotIDs3979.csv', 'fasta2csv_splitOnID_195340.csv', '.DS_Store', 'HUMAN_9606_idmapping_column1.txt', 'hist_canonicalNum_counts_in3979set_2504.csv', 'hist_canonicalNum_counts_in3991set_2507.csv', 'all3991_everLabeled_entries_simple.csv', 'isoform_counts_UKB_groupedby_stable_entry_fasta_source_173324.csv', 'HUMAN_9606_idmapping_2018_08_01download.dat', 'simple18432IDs.txt', 'canonical_NOT_1_IDs_in3991_56.csv', 'prism_hist_3991labeled_isoformCount.csv', 'prism_hist_10628ccds_canonicalIDnumber.csv', 'canonical_NOT_1_IDs_fastavsidmap_288.csv', 'HUMAN_9606_idmapping_column1_unique.txt', 'hist_canonicalNum_counts_multiIsoformOnly_10628.csv', 'merged_fasta_idat_can_iso_columnsFORnumber_isoforms_per_stableID

In [6]:
# isoform project filtering file down to subsets
prism = pd.read_csv("hist_canonical_isoformNum_all_stableEntries_multiIsoformFiltered_diffInISOnum_1_10628.csv")
print(prism.shape)
print(prism.head(4))

(10628, 10)
  stable_entry idat_isoformNumber fasta_isoformNumber  idat_len  fasta_len  \
0   A0A096LP49      ['1', '2', 0]            [0, '2']         3          2   
1   A0A140G945      ['1', '2', 0]            [0, '2']         3          2   
2   A0A1B0GVQ0      ['1', '2', 0]            [0, '2']         3          2   
3   A0A1B0GVS7      [0, '1', '2']            [0, '2']         3          2   

   labeledSet_3991  labeledShared_3979  UKBccds_9606_set_18433  \
0            False               False                    True   
1            False               False                   False   
2            False               False                   False   
3            False               False                   False   

   idatLen_minus_fastaLen  canonical_iso_number  
0                       1                     1  
1                       1                     1  
2                       1                     1  
3                       1                     1  


In [14]:
prism.head(10)
# output of isoform pro

,stable_entry,idat_isoformNumber,fasta_isoformNumber,idat_len,fasta_len,labeledSet_3991,labeledShared_3979,UKBccds_9606_set_18433,idatLen_minus_fastaLen,canonical_iso_number
0,A0A096LP49,"['1', '2', 0]","[0, '2']",3,2,False,False,True,1,1
1,A0A140G945,"['1', '2', 0]","[0, '2']",3,2,False,False,False,1,1
2,A0A1B0GVQ0,"['1', '2', 0]","[0, '2']",3,2,False,False,False,1,1
3,A0A1B0GVS7,"[0, '1', '2']","[0, '2']",3,2,False,False,False,1,1
4,A0AUZ9,"['1', '2', '3', '4', 0]","[0, '2', '3', '4']",5,4,False,False,True,1,1
5,A0AV02,"['1', '2', '3', '4', '5', 0]","[0, '2', '3', '4', '5']",6,5,False,False,True,1,1
6,A0AV96,"[0, '1', '2']","[0, '2']",3,2,False,False,True,1,1
7,A0AVF1,"['1', '2', '3', 0]","[0, '2', '3']",4,3,False,False,True,1,1
8,A0AVI2,"['1', '2', '4', 0]","[0, '2', '4']",4,3,False,False,True,1,1
9,A0AVI4,"['1', '2', 0]","[0, '2']",3,2,False,False,True,1,1


In [7]:
label = prism[prism['labeledSet_3991'] == True].copy()
ccds = prism[prism['UKBccds_9606_set_18433'] == True].copy()

In [8]:
print(ccds.shape)
print(label.shape)

(10301, 10)
(2507, 10)


# error with file name

In [12]:
label.to_csv("prism_hist_2507labeled_canonicalIDnumber.csv",index=False)
prism.to_csv("prism_hist_10628ccds_canonicalIDnumber.csv",index=False)

In [9]:
prism2 = pd.read_csv("isoform_counts_UKB_groupedby_stable_entry_fasta_source_173324.csv")
prism2.columns

Index(['stable_entry', 'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18433'],
      dtype='object')

In [10]:
label2 = prism2[prism2['labeledSet_3991'] == True].copy()
ccds2 = prism2[prism2['UKBccds_9606_set_18433'] == True].copy()

In [11]:
label2.to_csv("prism_hist_3991labeled_isoformCount.csv",index=False)
ccds2.to_csv("prism_hist_10628ccds_isoformCount.csv",index=False)

---
---
---

In [12]:
# split by -, group by stable ID
# splitUniprotID(dfukb, "ID")
def splitUniprotID(df, colname): 
    proIDsplit = df[colname].str.split("-", n=1, expand=True)
    proIDsplit.columns = ['stable_entry', 'isoformNumber']
    dfmerge = pd.concat([df,proIDsplit], axis=1)
    # converting canonical seq with no isofrom into 0 value
    dfmerge.isoformNumber.fillna(value=0, inplace=True)
    # converting isoform column into categorical
    dfmerge.isoformNumber = dfmerge.isoformNumber.astype('object')
    # idea, make the isoformnum and len in tuple, so i can compare if canonical 0 is -1 in ID mapping file
    stable_group = dfmerge.groupby(['stable_entry'])['isoformNumber'].apply(list).reset_index()
    return stable_group

# create list from stable_entry df for each column
def grabTrue_df(df, colname):
    dfTrue = df[df[colname] == 'True']
    dfTrue.reset_index(drop=True, inplace=True)
    return dfTrue

In [13]:
#dffasta = "UniprotKB_isoforms_canonical_filtered_homosapiens_9606_2018_06release.fasta"
#dfuni = fastaToDF(dffasta, "uniprot")
#dfuni.to_csv("fasta2csv_iso_canon_ukb_2018.csv",index=False)

dfuni=pd.read_csv("fasta2csv_isocanSequences_ukb2018_195341.csv")
# total lines = 195341
# extract column with ID and Sequence only

dfukb = dfuni[['ID','Length']].copy()
# parse ID column in proteome file

stable_fasta = splitUniprotID(dfukb, "ID")
describeMe(stable_fasta)

(173324, 2)
Index(['stable_entry', 'isoformNumber'], dtype='object')
stable_entry     0
isoformNumber    0
dtype: int64
(173324, 2)
Index(['stable_entry', 'isoformNumber'], dtype='object')
stable_entry     0
isoformNumber    0
dtype: int64


In [15]:
# 1st file read in 
dfuni.head(4)

,ID,entryName,Length,proSequence
0,O95139,NDUB6_HUMAN,128,MTGYTPDEKLRLQQLRELRRRWLKDQELSPREPVLPPQKMGPMEKF...
1,O95139-2,NDUB6_HUMAN,113,MTGYTPDEKLRLQQLRELRRRWLKDQELSPREPVLPPQKMGPMEKF...
2,O75438,NDUB1_HUMAN,58,MVNLLQIVRDHWVHVLVPMGFVIGCYLDRKSDERLTAFRNKSMLFK...
3,O75438-2,NDUB1_HUMAN,105,MICWRHPSAPCGRGEWQVPRSQLPLARVEFPVALGLGVAVGAEAAA...


In [16]:
dfukb.head(3)

,ID,Length
0,O95139,128
1,O95139-2,113
2,O75438,58


In [18]:
stable_fasta.head(2)

,stable_entry,isoformNumber
0,A0A023HHK9,[0]
1,A0A023HHL0,[0]


In [ ]:
# read in IDdat file
idat = pd.read_table("HUMAN_9606_idmapping_column1_unique.txt", header=None)
idat.columns = ['ID']

# parse ID column in proteome file
stable_idat = splitUniprotID(idat, "ID")
describeMe(stable_idat)

# add additional column if uniprot ID are in ccds set
stable_idat.sort_values(by=['stable_entry'], inplace = True)
stable_fasta.sort_values(by=['stable_entry'], inplace = True)

## splitting ID from Q9BVC4-4 to Q9BVC4 [4]

In [22]:
idat.sample(4)

,ID
115134,B9ECT3
152202,S4TZJ6
61617,A0A0U1RQF7
46431,A0A0A7C4Z2


In [23]:
stable_idat.head(3)

,stable_entry,isoformNumber
0,A0A023HHK9,[0]
1,A0A023HHL0,[0]
2,A0A023HJ61,[0]


In [24]:
stable_fasta.head(3)

,stable_entry,isoformNumber
0,A0A023HHK9,[0]
1,A0A023HHL0,[0]
2,A0A023HJ61,[0]


# the entries that only have 1 sequence are not included in canonical analysis
- so all IDs with isoform number of [0]

In [25]:
# changing column names preparing for merge of all rows and subset ccds rows
fasta = stable_fasta[['stable_entry','isoformNumber']].copy()
fasta.columns = ['stable_entry', 'fasta_isoformNumber']
idat = stable_idat[['stable_entry','isoformNumber']].copy()
idat.columns = ['stable_entry', 'idat_isoformNumber']

In [26]:
# creating merge on stable entry
merge170k = pd.merge(idat,fasta,how='inner',on=['stable_entry'])

In [27]:
# add length column to this merged file before parsing out TRUE ccds subset rows
merge170k['idat_len'] = merge170k['idat_isoformNumber'].str.len()
merge170k['fasta_len'] = merge170k['fasta_isoformNumber'].str.len()

In [28]:
# SUB SET LABELING
# reading in 18432 simple labels
ccds = pd.read_csv("simple18432IDs.txt",header=None, names = ['ID'])
labeled = pd.read_csv("all3991_everLabeled_entries_simple.csv")
sharedlabeled = pd.read_csv("uniprotIDs3979.csv", header=None, names = ['ID'])
# making lists
lsccds = ccds.ID.tolist() # in 18433 from which come the labeled subsets
lsccds.remove('ID') # len now 18432
labeled.columns = ['ID']
lslabeled = labeled.ID.tolist() # labeled before
lssharedlabeled = sharedlabeled.ID.tolist() # shared and labeled before

In [29]:
# $$$ labeling this df with the list of made to represent eachs subset
merge170k = addcolumnconditional(lslabeled, merge170k, 'stable_entry', 'labeledSet_3991')


In [30]:
# $$$
merge170k = addcolumnconditional(lssharedlabeled, merge170k, 'stable_entry', 'labeledShared_3979')


In [31]:
# $$$
merge170k = addcolumnconditional(lsccds, merge170k, 'stable_entry', 'UKBccds_9606_set_18433')

## POST labeling rows based on stable_entry col

In [32]:
merge170k.head(4)

,stable_entry,idat_isoformNumber,fasta_isoformNumber,idat_len,fasta_len,labeledSet_3991,labeledShared_3979,UKBccds_9606_set_18433
0,A0A023HHK9,[0],[0],1,1,False,False,False
1,A0A023HHL0,[0],[0],1,1,False,False,False
2,A0A023HJ61,[0],[0],1,1,False,False,False
3,A0A023HN28,[0],[0],1,1,False,False,False


In [33]:
checkColumnValues(merge170k, 'labeledSet_3991') # all IDs present
checkColumnValues(merge170k, 'labeledShared_3979') # all IDs present
checkColumnValues(merge170k, 'UKBccds_9606_set_18433') # all IDs present
#merge170k.to_csv("merged_fasta_idat_can_iso_columnsFORnumber_isoforms_per_stableID_173324.csv", index=False)
# ----------------------------------------------------------------------
# saving single isoform entries
# singleIsoform = merge170k[(merge170k['idat_len'] == 1) & (merge170k['fasta_len'] ==1)]
# checkColumnValues(singleIsoform,"idat_isoformNumber" )
# checkColumnValues(singleIsoform,"fasta_isoformNumber" )
# checkColumnValues(singleIsoform,"labeledSet_3991" )
# checkColumnValues(singleIsoform,"labeledShared_3979" )
# checkColumnValues(singleIsoform,"UKBccds_9606_set_18432" )
# describeMe(singleIsoform)
#singleIsoform.to_csv("merged_fasta_idat_singleIsoform_entries_162688.csv", index=False)

  labeledSet_3991   Count
0           False  169333
1            True    3991
  labeledShared_3979   Count
0              False  169345
1               True    3979
  UKBccds_9606_set_18433   Count
0                  False  154892
1                   True   18432


In [34]:
merge170k.head(4)

,stable_entry,idat_isoformNumber,fasta_isoformNumber,idat_len,fasta_len,labeledSet_3991,labeledShared_3979,UKBccds_9606_set_18433
0,A0A023HHK9,[0],[0],1,1,False,False,False
1,A0A023HHL0,[0],[0],1,1,False,False,False
2,A0A023HJ61,[0],[0],1,1,False,False,False
3,A0A023HN28,[0],[0],1,1,False,False,False


# comparing diff value counts between idat file and fasta_isoform file, its clear that ...
1) the idat file includes isoform details for all entry unless entry only has 1 seq. and 

2) fasta file includes isoform number for all non-canonical sequences.

In [35]:
checkColumnValues(merge170k, 'idat_isoformNumber') # all IDs present
checkColumnValues(merge170k, 'fasta_isoformNumber') # all IDs present

                                    idat_isoformNumber   Count
0                                                  [0]  162689
1                                            [0, 1, 2]    2619
2                                            [1, 2, 0]    2619
3                                         [0, 1, 2, 3]    1205
4                                         [1, 2, 3, 0]    1157
5                                      [0, 1, 2, 3, 4]     565
6                                      [1, 2, 3, 4, 0]     535
7                                   [1, 2, 3, 4, 5, 0]     296
8                                   [0, 1, 2, 3, 4, 5]     263
9                                [0, 1, 2, 3, 4, 5, 6]     156
10                               [1, 2, 3, 4, 5, 6, 0]     144
11                            [1, 2, 3, 4, 5, 6, 7, 0]      86
12                            [0, 1, 2, 3, 4, 5, 6, 7]      82
13                                           [1, 3, 0]      61
14                                           [0, 1, 3] 

# example protein: 



In [18]:
# pull out all non single Isoform rows
multiIsoform = merge170k[merge170k['idat_len'] != 1]
describeMe(multiIsoform) # 10635 rows
# checkColumnValues(multiIsoform,'idat_isoformNumber')
# checkColumnValues(multiIsoform,'fasta_isoformNumber')
# checkColumnValues(multiIsoform,"idat_isoformNumber")
# checkColumnValues(multiIsoform,"fasta_isoformNumber")
# checkColumnValues(multiIsoform,"labeledSet_3991")
# checkColumnValues(multiIsoform,"labeledShared_3979")
# checkColumnValues(multiIsoform,"UKBccds_9606_set_18432")
# getting difference in Isoform numbers from idat - fasta
multiIsoform = multiIsoform.assign(idatLen_minus_fastaLen = multiIsoform["idat_len"] - multiIsoform["fasta_len"])
# checkColumnValues(multiIsoform, 'idatLen_minus_fastaLen')
# 10628 had 1 difference, only keep these rows


(10635, 8)
Index(['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18433'],
      dtype='object')
stable_entry              0
idat_isoformNumber        0
fasta_isoformNumber       0
idat_len                  0
fasta_len                 0
labeledSet_3991           0
labeledShared_3979        0
UKBccds_9606_set_18433    0
dtype: int64


In [19]:
multiIsoform = multiIsoform[multiIsoform['idatLen_minus_fastaLen']==1]
describeMe(multiIsoform)
# checkColumnValues(multiIsoform,"labeledSet_3991")
# checkColumnValues(multiIsoform,"labeledShared_3979")
# checkColumnValues(multiIsoform,"UKBccds_9606_set_18432")
#multiIsoform.to_csv("merged_fasta_idat_multiIsoform_1moreIDinIDATfiltered_10628.csv", index=False)

(10628, 9)
Index(['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18433', 'idatLen_minus_fastaLen'],
      dtype='object')
stable_entry              0
idat_isoformNumber        0
fasta_isoformNumber       0
idat_len                  0
fasta_len                 0
labeledSet_3991           0
labeledShared_3979        0
UKBccds_9606_set_18433    0
idatLen_minus_fastaLen    0
dtype: int64


In [20]:
# FIGURE CODE for isoform count
KBisoforms = merge170k[['stable_entry','fasta_len','labeledSet_3991','labeledShared_3979','UKBccds_9606_set_18433']].copy()
KBisoforms.shape
#KBisoforms.to_csv("isoformCounts_UKBstable_entry_fastawIsos_173324.csv",index=False)

(173324, 5)

In [21]:
multiIsoform.reset_index(drop=True, inplace=True)

In [22]:
multiIsoform.head(10)

,stable_entry,idat_isoformNumber,fasta_isoformNumber,idat_len,fasta_len,labeledSet_3991,labeledShared_3979,UKBccds_9606_set_18433,idatLen_minus_fastaLen
0,A0A096LP49,"[1, 2, 0]","[0, 2]",3,2,False,False,True,1
1,A0A140G945,"[1, 2, 0]","[0, 2]",3,2,False,False,False,1
2,A0A1B0GVQ0,"[1, 2, 0]","[0, 2]",3,2,False,False,False,1
3,A0A1B0GVS7,"[0, 1, 2]","[0, 2]",3,2,False,False,False,1
4,A0AUZ9,"[1, 2, 3, 4, 0]","[0, 2, 3, 4]",5,4,False,False,True,1
5,A0AV02,"[1, 2, 3, 4, 5, 0]","[0, 2, 3, 4, 5]",6,5,False,False,True,1
6,A0AV96,"[0, 1, 2]","[0, 2]",3,2,False,False,True,1
7,A0AVF1,"[1, 2, 3, 0]","[0, 2, 3]",4,3,False,False,True,1
8,A0AVI2,"[1, 2, 4, 0]","[0, 2, 4]",4,3,False,False,True,1
9,A0AVI4,"[1, 2, 0]","[0, 2]",3,2,False,False,True,1


In [23]:
sidat = multiIsoform.idat_isoformNumber
sfas = multiIsoform.fasta_isoformNumber
print(sidat[2])
print(sfas[2])
c = list(set(sidat[2]) - set(sfas[2]))
print(c)

['1', '2', 0]
[0, '2']
['1']


In [24]:
# getting canonical isoform number
DAT = []
FAS = []
CAN = []
for i in range(len(sidat)):  
    lsIdat = sidat[i]
    lsFas = sfas[i]
    DAT.append(lsIdat)
    FAS.append(lsFas)
    dif = list(set(lsIdat) - set(lsFas))
    CAN.append(dif)

In [25]:
#idat_col = pd.DataFrame(DAT, columns = ['str_idat_isoformNumber'])
#fasta_col = pd.DataFrame(FAS, columns = ['str_fasta_isofo'])
canon_col = pd.DataFrame(CAN, columns = ['canonical_iso_number'])

In [26]:
final = pd.concat([multiIsoform, canon_col], axis=1)


In [27]:
print(multiIsoform.shape)
print(final.shape)

(10628, 9)
(10628, 10)


In [28]:
final.sample(14)

,stable_entry,idat_isoformNumber,fasta_isoformNumber,idat_len,fasta_len,labeledSet_3991,labeledShared_3979,UKBccds_9606_set_18433,idatLen_minus_fastaLen,canonical_iso_number
3499,Q13164,"[1, 2, 3, 4, 0]","[0, 2, 3, 4]",5,4,False,False,True,1,1
7557,Q96EP0,"[0, 1, 2, 3]","[0, 2, 3]",4,3,True,True,True,1,1
9953,Q9UJF2,"[1, 2, 0]","[0, 2]",3,2,True,True,True,1,1
2858,P56270,"[0, 1, 2, 3, 4]","[0, 2, 3, 4]",5,4,False,False,True,1,1
4062,Q16831,"[0, 1, 2]","[0, 2]",3,2,True,True,True,1,1
9315,Q9NS85,"[1, 2, 0]","[0, 2]",3,2,False,False,True,1,1
1357,P04746,"[1, 2, 0]","[0, 2]",3,2,False,False,True,1,1
476,O15164,"[0, 1, 2]","[0, 2]",3,2,True,True,True,1,1
3652,Q13887,"[1, 2, 3, 4, 0]","[0, 2, 3, 4]",5,4,False,False,True,1,1
10194,Q9UPQ0,"[0, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0, 2, 3, 4, 5, 6, 7, 8, 9, 10]",11,10,True,True,True,1,1


In [29]:
final.shape

(10628, 10)

In [31]:
final.to_csv("hist_canonical_isoformNum_all_stableEntries_multiIsoformFiltered_diffInISOnum_1_10628.csv", index=False)

In [30]:
checkColumnValues(final, 'canonical_iso_number')


  canonical_iso_number  Count
0                    1  10340
1                    2    104
2                    3     98
3                    4     49
4                    5     22
5                    6      5
6                    7      5
7                   10      2
8                    9      2
9                   11      1


In [39]:
canonicalcount = final.canonical_iso_number.value_counts()

1     10340
2       104
3        98
4        49
5        22
7         5
6         5
9         2
10        2
11        1
Name: canonical_iso_number, dtype: int64

In [40]:
canonicalcount.to_csv("hist_canonical_num_counts_fasta2018_idmap2018.csv", index=False)

In [43]:
failed = final[final['canonical_iso_number'] != '1']
failed.reset_index(inplace=True, drop=True)
failed.head(10)

,stable_entry,idat_isoformNumber,fasta_isoformNumber,idat_len,fasta_len,labeledSet_3991,labeledShared_3979,UKBccds_9606_set_18433,idatLen_minus_fastaLen,canonical_iso_number
0,A1Z1Q3,"[1, 2, 4, 5, 6, 0]","[0, 1, 4, 5, 6]",6,5,False,False,True,1,2
1,A2RUB1,"[0, 1, 2, 4, 5]","[0, 2, 1, 5]",5,4,False,False,True,1,4
2,A2VCL2,"[0, 1, 2, 3]","[0, 1, 2]",4,3,False,False,False,1,3
3,A6NCC3,"[0, 1, 2]","[0, 1]",3,2,False,False,True,1,2
4,A6NFD8,"[3, 4, 0]","[0, 4]",3,2,False,False,True,1,3
5,A6NFN9,"[1, 2, 3, 0]","[0, 1, 3]",4,3,False,False,True,1,2
6,O00139,"[0, 1, 2, 3, 4, 5]","[0, 1, 2, 4, 5]",6,5,True,True,True,1,3
7,O00327,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0, 1, 3, 4, 5, 6, 7, 8, 9]",10,9,False,False,True,1,2
8,O14522,"[0, 1, 3]","[0, 1]",3,2,False,False,True,1,3
9,O15021,"[0, 2, 3, 4, 5, 6]","[0, 2, 4, 3, 6]",6,5,False,False,True,1,5


In [48]:
failed.columns = ['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18432', 'idatLen_minus_fastaLen',
       'canonical_iso_number']
failed.to_csv("canonical_NOT_1_IDs_fastavsidmap_288.csv", index=False)

In [49]:
# create list from stable_entry df for each column
def grabTrue_df(df, colname):
    dfTrue = df[df[colname] == 'True']
    dfTrue.reset_index(drop=True, inplace=True)
    return dfTrue

In [54]:
labTRUE = grabTrue_df(final, 'labeledSet_3991')
labenspTRUE = grabTrue_df(final,'labeledShared_3979')
# should be 18432 shape
print(labTRUE.shape)
print(labenspTRUE.shape)

(2507, 10)
(2504, 10)


In [55]:
in3991 = labTRUE.canonical_iso_number.value_counts()
in3979 = labenspTRUE.canonical_iso_number.value_counts()

in91 = pd.DataFrame(in3991)
in79 = pd.DataFrame(in3979)




In [57]:
in91.reset_index(inplace=True)
in79.reset_index(inplace=True)


,index,canonical_iso_number
0,1,2451
1,2,22
2,3,17
3,4,10
4,5,4
5,10,1
6,7,1
7,6,1


In [58]:
in91.columns = ['Isoform_number','Count']
in79.columns = ['Isoform_number','Count']

In [65]:
in91.to_csv("hist_in3991_2507.csv",index=False)
in79.to_csv("hist_in3979_2504.csv", index=False)

In [61]:
not1_3991 = labTRUE[labTRUE['canonical_iso_number'] != '1']
not1_3979 = labenspTRUE[labenspTRUE['canonical_iso_number'] != '1']
print(not1_3979.shape)
print(not1_3991.shape)

(56, 10)
(56, 10)


In [66]:
not1_3991.to_csv("canonical_NOT_1_IDs_in3991_56.csv",index=False)
not1_3979.to_csv("canonical_NOT_1_IDs_in3979_56.csv",index=False)


# NOTES 

```python

singleIsoform = merge170k[(merge170k['idat_len'] == 1) & (merge170k['fasta_len'] ==1)]
checkColumnValues(singleIsoform,"idat_isoformNumber" )
checkColumnValues(singleIsoform,"fasta_isoformNumber" )
checkColumnValues(singleIsoform,"labeledSet_3991" )
checkColumnValues(singleIsoform,"labeledShared_3979" )
checkColumnValues(singleIsoform,"UKBccds_9606_set_18432" )
describeMe(singleIsoform)

  idat_isoformNumber   Count
0                [0]  162688
  fasta_isoformNumber   Count
0                 [0]  162688
  labeledSet_3991   Count
0           False  161205
1            True    1483
  labeledShared_3979   Count
0              False  161214
1               True    1474
  UKBccds_9606_set_18432   Count
0                  False  154563
1                   True    8125
(162688, 8)
Index(['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18432'],
      dtype='object')
stable_entry              0
idat_isoformNumber        0
fasta_isoformNumber       0
idat_len                  0
fasta_len                 0
labeledSet_3991           0
labeledShared_3979        0
UKBccds_9606_set_18432    0
dtype: int64
    
      idat_isoformNumber   Count
0                [0]  162688
  fasta_isoformNumber   Count
0                 [0]  162688
  labeledSet_3991   Count
0           False  161205
1            True    1483
  labeledShared_3979   Count
0              False  161214
1               True    1474
  UKBccds_9606_set_18432   Count
0                  False  154563
1                   True    8125
(162688, 8)
Index(['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18432'],
      dtype='object')
stable_entry              0
idat_isoformNumber        0
fasta_isoformNumber       0
idat_len                  0
fasta_len                 0
labeledSet_3991           0
labeledShared_3979        0
UKBccds_9606_set_18432    0
dtype: int64
    
    
#multiIsoform = merge170k[merge170k['idat_len'] != 1]
describeMe(multiIsoform)
checkColumnValues(multiIsoform,'idat_isoformNumber')
checkColumnValues(multiIsoform,'fasta_isoformNumber')
checkColumnValues(multiIsoform,"idat_isoformNumber" )
checkColumnValues(multiIsoform,"fasta_isoformNumber" )
checkColumnValues(multiIsoform,"labeledSet_3991" )
checkColumnValues(multiIsoform,"labeledShared_3979" )
checkColumnValues(multiIsoform,"UKBccds_9606_set_18432" )


(10635, 8)
Index(['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18432'],
      dtype='object')
stable_entry              0
idat_isoformNumber        0
fasta_isoformNumber       0
idat_len                  0
fasta_len                 0
labeledSet_3991           0
labeledShared_3979        0
UKBccds_9606_set_18432    0
dtype: int64
                                    idat_isoformNumber  Count
0                                            [1, 2, 0]   2619
1                                            [0, 1, 2]   2619
2                                         [0, 1, 2, 3]   1205
3                                         [1, 2, 3, 0]   1157
4                                      [0, 1, 2, 3, 4]    565
5                                      [1, 2, 3, 4, 0]    535
6                                   [1, 2, 3, 4, 5, 0]    296
7                                   [0, 1, 2, 3, 4, 5]    263
8                                [0, 1, 2, 3, 4, 5, 6]    156
9                                [1, 2, 3, 4, 5, 6, 0]    144
10                            [1, 2, 3, 4, 5, 6, 7, 0]     86
11                            [0, 1, 2, 3, 4, 5, 6, 7]     82
12                                           [1, 3, 0]     61
13                                           [0, 1, 3]     50
14                                              [1, 0]     47
15                                              [0, 1]     45
16                         [1, 2, 3, 4, 5, 6, 7, 8, 0]     45
17                         [0, 1, 2, 3, 4, 5, 6, 7, 8]     44
18                                        [1, 3, 4, 0]     29
19                                        [0, 1, 3, 4]     29
20                                        [0, 1, 2, 4]     28
21                      [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]     26
22                                        [1, 2, 4, 0]     25
23                                     [1, 2, 4, 5, 0]     21
24                      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]     21
25                                           [1, 4, 0]     15
26                                     [1, 3, 4, 5, 0]     14
27                                           [2, 3, 0]     12
28                                     [0, 1, 2, 4, 5]     11
29                                     [0, 1, 2, 3, 5]     11
..                                                 ...    ...
188                        [0, 1, 2, 3, 4, 5, 6, 7, 9]      1
189                [10, 12, 0, 1, 2, 3, 4, 5, 6, 8, 9]      1
190                        [1, 2, 3, 4, 6, 7, 8, 9, 0]      1
191  [10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9...      1
192                                          [5, 6, 0]      1
193  [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 11, 12, 13,...      1
194       [0, 10, 11, 12, 15, 16, 1, 2, 5, 6, 7, 8, 9]      1
195  [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 11, 12, 13,...      1
196                    [10, 0, 1, 3, 4, 5, 6, 7, 8, 9]      1
197                              [0, 1, 2, 4, 6, 7, 8]      1
198                                    [0, 3, 4, 5, 6]      1
199                                    [1, 3, 5, 7, 0]      1
200        [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 0]      1
201                                          [2, 5, 0]      1
202                    [0, 10, 1, 2, 3, 4, 6, 7, 8, 9]      1
203                                 [0, 1, 2, 3, 6, 7]      1
204                                    [0, 1, 4, 5, 7]      1
205                             [0, 1, 2, 5, 6, 8, 10]      1
206                                    [0, 2, 3, 4, 6]      1
207                                 [0, 1, 2, 4, 6, 7]      1
208     [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 11, 12, 13]      1
209                                       [4, 5, 6, 0]      1
210  [10, 11, 12, 13, 15, 16, 1, 2, 3, 4, 5, 6, 7, ...      1
211  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2...      1
212                           [1, 2, 3, 4, 6, 7, 8, 0]      1
213                                   [1, 2, 9, 0, 10]      1
214  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2...      1
215                            [1, 3, 7, 9, 0, 10, 13]      1
216               [10, 11, 12, 1, 3, 4, 5, 6, 8, 9, 0]      1
217                           [1, 2, 3, 4, 5, 6, 9, 0]      1

[218 rows x 2 columns]
                                   fasta_isoformNumber  Count
0                                               [0, 2]   5222
1                                            [0, 2, 3]   2304
2                                         [0, 2, 3, 4]   1048
3                                      [0, 2, 3, 4, 5]    518
4                                   [0, 2, 3, 4, 5, 6]    273
5                                [0, 2, 3, 4, 5, 6, 7]    160
6                                               [0, 3]    118
7                                                  [0]     92
8                             [0, 2, 3, 4, 5, 6, 7, 8]     86
9                                            [0, 3, 4]     56
10                                           [0, 2, 4]     56
11                         [0, 2, 3, 4, 5, 6, 7, 8, 9]     41
12                                              [0, 1]     33
13                                              [0, 4]     32
14                                        [0, 2, 4, 5]     29
15                                           [0, 1, 2]     27
16                     [0, 2, 3, 4, 5, 6, 7, 8, 9, 10]     26
17                                           [0, 1, 3]     26
18                 [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]     20
19                                        [0, 2, 3, 5]     20
20                                        [0, 3, 4, 5]     20
21                                     [0, 2, 4, 5, 6]     15
22             [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]     15
23                                        [0, 1, 2, 3]     13
24         [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]     12
25                                        [0, 1, 3, 4]     10
26                                        [0, 2, 4, 3]     10
27                                     [0, 2, 3, 4, 6]      9
28                                     [0, 2, 3, 5, 6]      8
29                                     [0, 1, 3, 4, 5]      8
..                                                 ...    ...
183                                 [0, 2, 1, 4, 5, 6]      1
184                       [0, 2, 3, 4, 5, 6, 7, 10, 8]      1
185                                 [0, 3, 2, 4, 5, 6]      1
186                              [0, 2, 3, 4, 5, 6, 9]      1
187  [0, 2, 3, 4, 9, 10, 11, 12, 13, 14, 15, 16, 5,...      1
188  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      1
189                                       [0, 2, 1, 3]      1
190                                 [0, 1, 2, 6, 7, 8]      1
191                [0, 2, 6, 5, 4, 8, 3, 7, 9, 10, 11]      1
192                  [0, 3, 4, 8, 9, 5, 6, 10, 11, 12]      1
193                   [0, 2, 3, 4, 5, 6, 8, 9, 10, 11]      1
194                                 [0, 3, 4, 5, 6, 7]      1
195                       [0, 2, 3, 4, 5, 6, 7, 9, 10]      1
196                                       [0, 2, 5, 4]      1
197                                 [0, 2, 8, 4, 9, 1]      1
198                                       [0, 3, 5, 7]      1
199           [0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13]      1
200                                 [0, 2, 3, 4, 6, 5]      1
201                       [0, 2, 3, 4, 7, 6, 8, 9, 10]      1
202                                    [0, 2, 3, 6, 7]      1
203                                       [0, 4, 5, 7]      1
204                                [0, 2, 5, 6, 8, 10]      1
205                                       [0, 2, 6, 1]      1
206                                    [0, 2, 4, 6, 7]      1
207  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...      1
208  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      1
209                                      [0, 2, 9, 10]      1
210  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      1
211                               [0, 3, 7, 9, 10, 13]      1
212  [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...      1

[213 rows x 2 columns]
                                    idat_isoformNumber  Count
0                                            [1, 2, 0]   2619
1                                            [0, 1, 2]   2619
2                                         [0, 1, 2, 3]   1205
3                                         [1, 2, 3, 0]   1157
4                                      [0, 1, 2, 3, 4]    565
5                                      [1, 2, 3, 4, 0]    535
6                                   [1, 2, 3, 4, 5, 0]    296
7                                   [0, 1, 2, 3, 4, 5]    263
8                                [0, 1, 2, 3, 4, 5, 6]    156
9                                [1, 2, 3, 4, 5, 6, 0]    144
10                            [1, 2, 3, 4, 5, 6, 7, 0]     86
11                            [0, 1, 2, 3, 4, 5, 6, 7]     82
12                                           [1, 3, 0]     61
13                                           [0, 1, 3]     50
14                                              [1, 0]     47
15                                              [0, 1]     45
16                         [1, 2, 3, 4, 5, 6, 7, 8, 0]     45
17                         [0, 1, 2, 3, 4, 5, 6, 7, 8]     44
18                                        [1, 3, 4, 0]     29
19                                        [0, 1, 3, 4]     29
20                                        [0, 1, 2, 4]     28
21                      [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]     26
22                                        [1, 2, 4, 0]     25
23                                     [1, 2, 4, 5, 0]     21
24                      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]     21
25                                           [1, 4, 0]     15
26                                     [1, 3, 4, 5, 0]     14
27                                           [2, 3, 0]     12
28                                     [0, 1, 2, 4, 5]     11
29                                     [0, 1, 2, 3, 5]     11
..                                                 ...    ...
188                        [0, 1, 2, 3, 4, 5, 6, 7, 9]      1
189                [10, 12, 0, 1, 2, 3, 4, 5, 6, 8, 9]      1
190                        [1, 2, 3, 4, 6, 7, 8, 9, 0]      1
191  [10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9...      1
192                                          [5, 6, 0]      1
193  [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 11, 12, 13,...      1
194       [0, 10, 11, 12, 15, 16, 1, 2, 5, 6, 7, 8, 9]      1
195  [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 11, 12, 13,...      1
196                    [10, 0, 1, 3, 4, 5, 6, 7, 8, 9]      1
197                              [0, 1, 2, 4, 6, 7, 8]      1
198                                    [0, 3, 4, 5, 6]      1
199                                    [1, 3, 5, 7, 0]      1
200        [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 0]      1
201                                          [2, 5, 0]      1
202                    [0, 10, 1, 2, 3, 4, 6, 7, 8, 9]      1
203                                 [0, 1, 2, 3, 6, 7]      1
204                                    [0, 1, 4, 5, 7]      1
205                             [0, 1, 2, 5, 6, 8, 10]      1
206                                    [0, 2, 3, 4, 6]      1
207                                 [0, 1, 2, 4, 6, 7]      1
208     [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 10, 11, 12, 13]      1
209                                       [4, 5, 6, 0]      1
210  [10, 11, 12, 13, 15, 16, 1, 2, 3, 4, 5, 6, 7, ...      1
211  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2...      1
212                           [1, 2, 3, 4, 6, 7, 8, 0]      1
213                                   [1, 2, 9, 0, 10]      1
214  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2...      1
215                            [1, 3, 7, 9, 0, 10, 13]      1
216               [10, 11, 12, 1, 3, 4, 5, 6, 8, 9, 0]      1
217                           [1, 2, 3, 4, 5, 6, 9, 0]      1

[218 rows x 2 columns]
                                   fasta_isoformNumber  Count
0                                               [0, 2]   5222
1                                            [0, 2, 3]   2304
2                                         [0, 2, 3, 4]   1048
3                                      [0, 2, 3, 4, 5]    518
4                                   [0, 2, 3, 4, 5, 6]    273
5                                [0, 2, 3, 4, 5, 6, 7]    160
6                                               [0, 3]    118
7                                                  [0]     92
8                             [0, 2, 3, 4, 5, 6, 7, 8]     86
9                                            [0, 3, 4]     56
10                                           [0, 2, 4]     56
11                         [0, 2, 3, 4, 5, 6, 7, 8, 9]     41
12                                              [0, 1]     33
13                                              [0, 4]     32
14                                        [0, 2, 4, 5]     29
15                                           [0, 1, 2]     27
16                     [0, 2, 3, 4, 5, 6, 7, 8, 9, 10]     26
17                                           [0, 1, 3]     26
18                 [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]     20
19                                        [0, 2, 3, 5]     20
20                                        [0, 3, 4, 5]     20
21                                     [0, 2, 4, 5, 6]     15
22             [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]     15
23                                        [0, 1, 2, 3]     13
24         [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]     12
25                                        [0, 1, 3, 4]     10
26                                        [0, 2, 4, 3]     10
27                                     [0, 2, 3, 4, 6]      9
28                                     [0, 2, 3, 5, 6]      8
29                                     [0, 1, 3, 4, 5]      8
..                                                 ...    ...
183                                 [0, 2, 1, 4, 5, 6]      1
184                       [0, 2, 3, 4, 5, 6, 7, 10, 8]      1
185                                 [0, 3, 2, 4, 5, 6]      1
186                              [0, 2, 3, 4, 5, 6, 9]      1
187  [0, 2, 3, 4, 9, 10, 11, 12, 13, 14, 15, 16, 5,...      1
188  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      1
189                                       [0, 2, 1, 3]      1
190                                 [0, 1, 2, 6, 7, 8]      1
191                [0, 2, 6, 5, 4, 8, 3, 7, 9, 10, 11]      1
192                  [0, 3, 4, 8, 9, 5, 6, 10, 11, 12]      1
193                   [0, 2, 3, 4, 5, 6, 8, 9, 10, 11]      1
194                                 [0, 3, 4, 5, 6, 7]      1
195                       [0, 2, 3, 4, 5, 6, 7, 9, 10]      1
196                                       [0, 2, 5, 4]      1
197                                 [0, 2, 8, 4, 9, 1]      1
198                                       [0, 3, 5, 7]      1
199           [0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13]      1
200                                 [0, 2, 3, 4, 6, 5]      1
201                       [0, 2, 3, 4, 7, 6, 8, 9, 10]      1
202                                    [0, 2, 3, 6, 7]      1
203                                       [0, 4, 5, 7]      1
204                                [0, 2, 5, 6, 8, 10]      1
205                                       [0, 2, 6, 1]      1
206                                    [0, 2, 4, 6, 7]      1
207  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...      1
208  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      1
209                                      [0, 2, 9, 10]      1
210  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      1
211                               [0, 3, 7, 9, 10, 13]      1
212  [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...      1

[213 rows x 2 columns]
  labeledSet_3991  Count
0           False   8127
1            True   2508
  labeledShared_3979  Count
0              False   8130
1               True   2505
  UKBccds_9606_set_18432  Count
0                   True  10307
1                  False    328
      
      
      checkColumnValues(multiIsoform, 'idatLen_minus_fastaLen')
      
      
      idatLen_minus_fastaLen  Count
0                       1  10628
1                       0      6
2                      -1      1
      
      
      
multiIsoform = multiIsoform[multiIsoform['idatLen_minus_fastaLen']==1]
describeMe(multiIsoform)
checkColumnValues(multiIsoform,"labeledSet_3991")
checkColumnValues(multiIsoform,"labeledShared_3979")
checkColumnValues(multiIsoform,"UKBccds_9606_set_18432")
      
      
(10628, 9)
Index(['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18432', 'idatLen_minus_fastaLen'],
      dtype='object')
stable_entry              0
idat_isoformNumber        0
fasta_isoformNumber       0
idat_len                  0
fasta_len                 0
labeledSet_3991           0
labeledShared_3979        0
UKBccds_9606_set_18432    0
idatLen_minus_fastaLen    0
dtype: int64
  labeledSet_3991  Count
0           False   8121
1            True   2507
  labeledShared_3979  Count
0              False   8124
1               True   2504
  UKBccds_9606_set_18432  Count
0                   True  10301
1                  False    327
      
      
      
multiIsoform.to_csv("merged_fasta_idat_multiIsoform_1moreIDinIDATfiltered_10628.csv", index=False)

      
# how many rows in idat have 0 in the list, which means i can drop it
#multiIsoform['flag0_idat'] = multiIsoform.apply(lambda x: 0 in x['idat_isoformNumber'], axis=1).astype(int)
multiIsoform['flag0_fasta'] = multiIsoform.apply(lambda x: 0 in x['fasta_isoformNumber'], axis=1).astype(int)

checkColumnValues(multiIsoform, 'flag0_idat')
checkColumnValues(multiIsoform, 'flag0_fasta')
flag0_idat  Count
0           1  10628
   flag0_fasta  Count
0            1  10628

      
```

---
---
---

---
---
---
---
---
---

# initial attempt at canonical question on 19_09_23_UniProt_CANONICAL_counting_XTRA_ATTEMPTS.html 
- problem was using 2019 id mapping file
- also only considering in 2019 file the ENSP linked uniprot IDs which limitied power of idat to decipher canonical number